In [ ]:
from ftplib import FTP
import ftputil
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import pygrib as pg
import glob
import os
from pathlib import Path
from datetime import timedelta

In [ ]:
inpath = '/home/opn/script/siam-netcdf-generator/coba/tes/envyosa/arome/input'

In [ ]:
def filedownload(grbzname, maxretry : int=3,retryfreq : int = 3):      
    import time
    with ftputil.FTPHost("172.19.0.47", "transmet2023", "99xrAQbW2qN7h") as ftp_host:
        file = f'MODELAROME/{grbzname}'
        basename = os.path.basename(file)
        inpathbin = Path(f'{inpath}')
        fout = Path(inpathbin, basename)
        fout.parent.mkdir(parents=True, exist_ok=True)
    
        if fout.exists():
            return fout
        else:
            for i in range(maxretry):
                if ftp_host.path.exists(file):
                    ftp_host.download(file,fout)
                    print(f'file downloaded {fout}')
                    return fout
                else:
                    print(f'File {file} not found')
                time.sleep(retryfreq)
            print(f'File {file} not found after {maxretry} retries')
            pass

In [ ]:
filedownload('A_YMJA70LFPW141700_C_LFPW_20240214173302_arome_indo_20240214_1200_02.grib')

In [8]:
import glob
from pathlib import Path
outbin = Path('/mnt/data/input/arome/2024/02/20_12')
filename = glob.glob(f'{outbin}/*.grib')

In [9]:
filename.sort()

In [13]:
filename

['/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173055_arome_indo_20240220_1200_00.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173150_arome_indo_20240220_1200_01.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173250_arome_indo_20240220_1200_02.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173358_arome_indo_20240220_1200_03.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173513_arome_indo_20240220_1200_04.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173619_arome_indo_20240220_1200_05.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173728_arome_indo_20240220_1200_06.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173837_arome_indo_20240220_1200_07.grib',
 '/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173955_arome_ind

In [10]:
import xarray as xr
xr.set_options(keep_attrs=False)
import glob
ds = xr.open_mfdataset(filename[1:],engine="cfgrib",
                       backend_kwargs=dict(filter_by_keys={'shortName':'mcc'}),
                       concat_dim='valid_time', combine='nested', parallel=True)

In [ ]:
ds['mcc'][]

In [8]:
import numpy as np
np.shape(ds['efg10'].values)

(36, 1201, 2601)

In [9]:
dscape = xr.open_mfdataset(filename[1:],engine="cfgrib",
                       backend_kwargs=dict(filter_by_keys={'shortName':'CAPE_INS'}),
                       concat_dim='valid_time', combine='nested', parallel=True)

In [10]:
dscape

<xarray.Dataset>
Dimensions:     (latitude: 1201, longitude: 2601, valid_time: 36)
Coordinates:
    time        datetime64[ns] 2024-02-20T12:00:00
    step        (valid_time) timedelta64[ns] 01:00:00 ... 1 days 12:00:00
    level       float64 0.0
  * latitude    (latitude) float64 15.0 14.97 14.95 ... -14.95 -14.98 -15.0
  * longitude   (longitude) float64 85.0 85.03 85.05 85.08 ... 150.0 150.0 150.0
  * valid_time  (valid_time) datetime64[ns] 2024-02-20T13:00:00 ... 2024-02-22
Data variables:
    CAPE_INS    (valid_time, latitude, longitude) float32 dask.array<chunksize=(1, 1201, 2601), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          14
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2024-02-21T14:03 GRIB to CDM+CF via cfgrib-0.9.1...

In [28]:
dscc = xr.open_mfdataset(filename[1:],engine="cfgrib",
                       backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'isobaricInhPa','shortName':'cc'}),
                       concat_dim='valid_time', combine='nested', parallel=True)

In [29]:
dscc

,Array,Chunk
Bytes,9.64 GiB,274.08 MiB
Shape,"(36, 23, 1201, 2601)","(1, 23, 1201, 2601)"
Count,144 Tasks,36 Chunks
Type,float32,numpy.ndarray


In [30]:
dstke = xr.open_mfdataset(filename[1:],engine="cfgrib",
                       backend_kwargs=dict(filter_by_keys={'typeOfLevel': 'isobaricInhPa','shortName':'tke'}),
                       concat_dim='valid_time', combine='nested', parallel=True)

In [31]:
dstke

,Array,Chunk
Bytes,7.54 GiB,214.49 MiB
Shape,"(36, 18, 1201, 2601)","(1, 18, 1201, 2601)"
Count,144 Tasks,36 Chunks
Type,float32,numpy.ndarray


In [5]:
import pygrib as pg
grb = [grb for grb in pg.open(filename[0])]
grb[0]

1:Temperature:K (instant):regular_ll:surface:level 0:fcst time 14 hrs:from 202402201200

In [10]:
filename[0]

'/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220175023_arome_indo_20240220_1200_14.grib'

In [8]:
for file in filename:
    grb = [grb for grb in pg.open(file)]
    if grb[0].forecastTime == 0:
        initgrib = file

In [9]:
initgrib

'/mnt/data/input/arome/2024/02/20_12/A_YMJA70LFPW201700_C_LFPW_20240220173055_arome_indo_20240220_1200_00.grib'

In [ ]:
ds.sortby(ds.valid_time)

In [ ]:
grbs = [pg.open(x) for x in gribnames]

In [ ]:
grbs = grbs[0:21] + grbs[22:]

In [ ]:
grbs

In [ ]:
i = 0
for grbb in grbs:
    grbb.select(typeOfLevel = 'surface', shortName=['hcc','lcc','mcc','vis'])
    print('done'+str(i))
    i+=1

In [ ]:
surface = [grbb.select(typeOfLevel = 'surface', shortName=['hcc','lcc','mcc','vis']) for grbb in grbs]

In [ ]:
surface

In [ ]:
g21 = [grb for grb in grbs[0]]

In [ ]:
grbs[0].keys()

In [ ]:
surface = grbs.select(typeOfLevel = 'surface', shortName=['hcc','lcc','mcc','vis'])

In [ ]:
surface

In [ ]:
hrbs = pg.open(gribnames[30])

In [ ]:
hsurface = hrbs.select(typeOfLevel = 'surface', shortName=['hcc','lcc','mcc','vis'])

In [ ]:
hsurface

In [ ]:
ds.seek(0)
for dss in ds:
    print(dss)

In [ ]:
lat, lon = dss.latlons()

In [ ]:
np.shape(lat)

In [ ]:
np.shape(lon)

In [ ]:
filedownload('A_YMJA70LFPW141700_C_LFPW_20240214174301_arome_indo_20240214_1200_09.grib')

In [ ]:
import pygrib as pg
grbs = pg.open('input/A_YMJA70LFPW141700_C_LFPW_20240214174301_arome_indo_20240214_1200_09.grib')

In [ ]:
grbs.seek(0)
cc = grbs.select(typeOfLevel= 'isobaricInhPa', shortName='cc')

In [ ]:
grbs.seek(0)
t = grbs.select(typeOfLevel= 'isobaricInhPa', shortName='t')

In [ ]:
grbs.seek(0)
tke = grbs.select(typeOfLevel= 'isobaricInhPa', shortName='tke')

In [ ]:
grbs.seek(0)
vis = grbs.select(shortName='vis')

In [ ]:
vis

In [ ]:
cclev = [x.level for x in cc]

In [ ]:
cclev

In [ ]:
tkelev = [x.level for x in tke]
tkelev

In [ ]:
notin = [x not in tkelev for x in cclev]

In [ ]:
notin

In [ ]:
cc = isobaric.select(shortName='cc')
# t = 
# tke = 

In [ ]:
grbs.seek(0)
grbb = grbs.select(typeOfLevel=['isobaricInhPa','surface','heightAboveGround'])
grb = [x for x in grbs]
def sortName(e):
    return e['shortName']
grb.sort(key=sortName)
name = [[f'{x.shortName},{x.name},{x.typeOfLevel}'] for x in grbb]

In [ ]:
np.unique(name)

In [ ]:
grbs.seek(0)
cape = grbs.select(typeOfLevel='unknown',)

In [ ]:
cape

In [ ]:
ds = xr.open_mfdataset('input/A_YMJA70LFPW141700_C_LFPW_20240214173302_arome_indo_20240214_1200_02.grib',engine="cfgrib",
                       backend_kwargs=dict(filter_by_keys={'name': 'Pressure','typeOfLevel':'unknown'}),
                       concat_dim='valid_time', combine='nested', parallel=True)

In [ ]:
ds

In [ ]:
grbs.seek(0)
grb = [x for x in grbs]

In [ ]:
len(grb)

In [ ]:
import ftputil
import re
def filenamegen(dtn):
        pattern = f'_arome_indo_{dtn:%Y}{dtn:%m}{dtn:%d}'
        with ftputil.FTPHost("172.19.0.47", "transmet2023", "99xrAQbW2qN7h") as ftp_host:
                names = ftp_host.listdir('MODELAROME/')
                a = [x for x in names if re.search(pattern,x)]

In [ ]:
num = '20240218'
pattern = f'_arome_indo_{num}'
with ftputil.FTPHost("172.19.0.47", "transmet2023", "99xrAQbW2qN7h") as ftp_host:
        names = ftp_host.listdir('MODELAROME/')
        a = [x for x in names if re.search(pattern,x)]

In [11]:
import xarray as xr
ds1 = xr.open_dataset('/mnt/data/output/arome/arome_20240220_1200.nc')

In [23]:
ds1

<xarray.Dataset>
Dimensions:  (lat: 1201, level: 23, lon: 2601, time: 36, tkeime: 36)
Coordinates:
  * time     (time) datetime64[ns] 2024-02-20T13:00:00 ... 2024-02-22
  * level    (level) float64 1e+03 950.0 925.0 900.0 ... 200.0 175.0 150.0 100.0
  * lat      (lat) float64 15.0 14.97 14.95 14.92 ... -14.93 -14.95 -14.98 -15.0
  * lon      (lon) float64 85.0 85.03 85.05 85.08 ... 149.9 150.0 150.0 150.0
Dimensions without coordinates: tkeime
Data variables:
    cc       (time, level, lat, lon) float32 ...
    t        (time, level, lat, lon) float32 ...
    tke      (tkeime, level, lat, lon) float64 ...
    hcc      (time, lat, lon) float32 ...
    lcc      (time, lat, lon) float32 ...
    mcc      (time, lat, lon) float32 ...
    vis      (time, lat, lon) float32 ...
    efg10    (time, lat, lon) float32 ...
    nfg10    (time, lat, lon) float32 ...
    cape     (time, lat, lon) float32 ...

In [15]:
ds['mcc'].values

array([[[2.5999998e-09, 2.5999998e-09, 2.5999998e-09, ...,
         2.5999998e-09, 2.5999998e-09, 2.5999998e-09],
        [2.5999998e-09, 2.5999998e-09, 2.5999998e-09, ...,
         2.5999998e-09, 2.5999998e-09, 2.5999998e-09],
        [2.5999998e-09, 2.5999998e-09, 2.5999998e-09, ...,
         2.5999998e-09, 2.5999998e-09, 2.5999998e-09],
        ...,
        [3.0799999e+01, 3.1200001e+01, 3.0400000e+01, ...,
         2.5999998e-09, 2.5999998e-09, 2.5999998e-09],
        [2.1200001e+01, 2.2000000e+01, 2.2000000e+01, ...,
         2.5999998e-09, 2.5999998e-09, 2.5999998e-09],
        [1.3600000e+01, 1.4800000e+01, 1.5600000e+01, ...,
         2.5999998e-09, 2.5999998e-09, 2.5999998e-09]],

       [[2.5999998e-09, 2.5999998e-09, 2.5999998e-09, ...,
         4.0000001e-01, 4.0000001e-01, 4.0000001e-01],
        [2.5999998e-09, 2.5999998e-09, 2.5999998e-09, ...,
         4.0000001e-01, 4.0000001e-01, 4.0000001e-01],
        [2.5999998e-09, 2.5999998e-09, 2.5999998e-09, ...,
         4.000